# DataPrep do conjunto de navios
## Grupo 2 - Trabalho de Conclusão de Curso
### - Bruno Uchôa Brandão e Silva - 12412440
### - Rayan Luz Ralile - 12412502

===============================================================================

### Passo 1: carregamento das bibliotecas usadas no programa

In [1]:
import numpy as np
import scipy.signal
import librosa
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
import pickle

### Passo 2: construção das funções auxiliares

In [2]:
# Funções de salvar e carregar objetos diversos - otimizar os passos de tratamento do dataset e treino
def salva_objeto(obj, file_path = "data.pickle"):
    with open(file_path, 'wb') as file:
        pickle.dump(obj, file)
        
def carrega_objeto(file_path = "data.pickle"):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

### Passo 3: construção da função que lê os arquivos de áudio e aplica as técnicas descritas no trabalho para a transformação de domínio para o tempo-frequência e construção dos bins de frequência.

In [3]:
# Lê os arquivos de áudio e processa a técnica PSD
def compute_psd(file_path, center_time):
    audio_data, original_sample_rate = librosa.load(file_path, sr=None)
    # original_sample_rate -> 48000Hz
    # Decima os dados de áudio
    audio_data = librosa.resample(audio_data, orig_sr=original_sample_rate, target_sr=4800)
    sample_rate = 4800
    
    
    # Computar o início e fim em torno do tempo de PMA em máx de 10 minutos
    start_sample = max(0, int((center_time - 5 * 60) * sample_rate))
    end_sample = min(len(audio_data), int((center_time + 5 * 60) * sample_rate))

    # Verifica se o tempo está completo
    if end_sample - start_sample != 10 * 60 * sample_rate:
        raise ValueError("Verificar argumento center_time - não efetivou intervalo por inteiro")


    audio_data = audio_data[start_sample:end_sample]
    # Reshape a matriz para as 600 amostras de 1 segundo cada
    chunks = audio_data.reshape(600, sample_rate)

    # Computar a densidade espectral de frequência usando o método Welch do SciPy
    psd_list = []
    for chunk in chunks:
        freqs, psd = scipy.signal.welch(chunk, sample_rate, nperseg=480, noverlap=240)
        # Seleciona as baixas frequências, de 10Hz a 2000Hz
        psd = psd[(freqs >= 10) & (freqs <= 2000)]
        psd_list.append(psd)
    return np.array(psd_list)

### Passo 4: constrói os arquivos de bins de frequência com suas classificações (dataset). A classificação no MLP do Scikit-learn usa string, enquanto no MLP do Keras usa um número inteiro, motivo pelo qual ele vai gerar, na construção do dataset, dois arquivos distintos com a biblioteca pickle.

In [4]:
classes = {}
classes = {'Scikit': ["Navio Pequeno Porte", "Navio Pequeno Porte", "Navio Pequeno Porte", "Navio Pequeno Porte", 
                  "Navio Grande Porte", "Navio Grande Porte","Navio Grande Porte","Navio Grande Porte", 
                  "Navio Pequeno Porte","Navio Pequeno Porte", "Navio Grande Porte", "Navio Grande Porte", 
                  "Navio Pequeno Porte", "Navio Pequeno Porte", "Navio Pequeno Porte", "Navio Grande Porte", 
                  "Navio Pequeno Porte", "Navio Grande Porte","Navio Pequeno Porte"], 
           'Keras': [1, 1, 1, 1, 0, 0,0,0,1,1,0,0,1,1,1,0,1,0,1]}

arquivos_wav = []

folderETAS = '../ETAS_WAV/'

for i in range(1, 20):
    nome_arquivo = folderETAS + str(i) + ".wav"
    arquivos_wav.append(nome_arquivo)

### Passo 5: obtém hardcoded os tempos dos PMAs (Ponto de Maior Aproximação) em cada arquivo .wav para aplicar o raciocínio de treinar as redes com os pontos 5 minutos antes e depois do PMA

In [5]:
# agora a lista dos tempos de centro em segundos
arrayTempos = [1840, 1680, 1710, 1700, 1700, 1840, 1840, 1800, 1800, 1800, 1800, 1800, 1700, 1800, 1860,
              1800, 1800, 1800, 1800]

### Passo 6: construção dos datasets, tanto pro MLP do Scikit-learn quanto pro Keras

In [6]:
# Listas de Dataset e label
classifiers = ['Scikit','Keras']
for classifier in classifiers:
    dataset = []
    labels = []
    contador = 0
    # Loop através dos arquivos de áudio
    for audio_file_path, audio_class in zip(arquivos_wav, classes[classifier]):
        psd = compute_psd(audio_file_path, arrayTempos[contador])
        dataset.extend(psd)
        labels.extend([audio_class]*600)  # Repete a classificação para as 600 amostras - etiquetagem
        contador += 1

    # Converte para numpy
    dataset = np.array(dataset)
    labels = np.array(labels)

    # Normaliza dataset com o método Min-Max
    scaler = MinMaxScaler()
    dataset = scaler.fit_transform(dataset)
    salva_objeto(dataset,"dataset.pickle")
    if classifier == 'Keras':
        salva_objeto(labels,"labels_keras.pickle")
    else:
        salva_objeto(labels,"labels_scikit.pickle")